<center><font size = "10"> Week 2 - Morphologies <center>
<center><font size = "8">Tutorial 02: Looking for errors<center>

<font size ="3"><font color='blue'>Throughout this tutorial you will learn:
    
<font size ="3"><font color='blue'>- What is inside a morphology file
    
<font size ="3"><font color='blue'>- Check errors in a morphology file with NeuroM

# 1. Morphology files 

<font size ="3">To work with any type of file, a good practice is always to check what is inside the file and how the file is organized. In the case of morphology files, each file is broken up onto sections, i.e. cell body (soma), apical dendrite, basal dendtrite, axon etc. Each of these sections contains the points that make up the neuron's structure. There are two main types of format files: ASCII and SWC.
    
<font size ="3">- __ASCII files:__ the file is divided in different blocks (CellBody, Axon, Dendrite, Apical). Inside each block each line has 6 fileds encoding data for a single compartment. 
- the (x, y, z) coordinate points
- the compartment width 
- an integer number as compartment identifier
- parent compartment 
 
<font size ="3">Every structure that branches from the soma receives the “Root” or “R” designation. Then its children are given a designation of either “1” or “2”. Then these two branches will give rise to more branches whose designation is either a “1” or “2” also. By this way, a branch from the tree can be identified by a string of 1’s and 2’s that identify it in the hierarchy. Data sets are ended with several different phrases depending on how you end your splits. You can end a split normally, or incompletely. Finally, the end of a tree is specified once the end of a tree is reached and new trees are given the same numbering system. Therefore, branch numbering is unique only within its own tree and not to the entire set of trees. (Check the figure for better understanding)

<font size ="3">- __SWC files:__ SWC are the initials of the last names of E.W. Stockley, H.V. Wheal, and H.M. Cole, who developed a system for generating morphometric reconstructions of neurons. In this type of morphology file format each line has 7 fields encoding data for a single neuronal compartment: 
- an integer number as compartment identifier
- type of neuronal compartment: 0 - undefined, 1 - soma, 2 - axon, 3 - basal dendrite, 4 - apical dendrite, 5 - custom (user-defined preferences), 6 - unspecified neurites, 7 - glia processes.
- x coordinate of the compartment
- y coordinate of the compartment
- z coordinate of the compartment
- radius of the compartment
- parent compartment    
    
<font size ="3">Every compartment has only one parent and the parent compartment for the first point in each file is always -1 (if the file does not include the soma information then the originating point of the tree will be connected to a parent of -1). The index for parent compartments are always less than child compartments. Loops and unconnected branches are excluded. All trees should originate from the soma and have parent type 1 if the file includes soma information. Soma can be a single point or more than one point. When the soma is encoded as one line in the SWC, it is interpreted as a "sphere". When it is encoded by more than 1 line, it could be a set of tapering cylinders (as in some pyramidal cells) or even a 2D projected contour ("circumference"). [explanation [here](http://www.neuromorpho.org/myfaq.jsp)] (Check the figure for better understanding).

    
<br><img src="MorphologyFormatsExample.png" width="1000" height="1000">

<br><font size ="3">Let's open one of the morphology files ('good_morph.asc' or 'bad_morph.asc') and check how a real file looks like. You will worc with .swc format files later.

In [ ]:
# First let's define the path in which we have the cells to work with.
bad_neuron = '...path_to_bad_morph...'
good_neuron = '...path_to_good_morph...'

# Now we can open the file
f = open(good_neuron, 'r')
for line in f:
    print(repr(line))
    

# 2. Checking for mistakes
<font size ="3">During the lecture you have seen that it could happen that the reconstructed morpholgies have some errors that have to be solved in case we want to work with this morphology file. After seeing a real morphology file, you could imagine that checking for errors in the file is not an easy task.
    
<font size ="3">Fortunately, NeuroM has a package to help us with this issue called [neurom.check.runner](http://pydoc.net/neurom/1.4.5/neurom.check.neuron_checks/).

<font size ="3"> We are going to see how it works for "neuron_good.asc" and "neuron_bad.asc"

In [ ]:
# The structural checks concern the valid structure of a morphology.
# The neuron_checks are more specific to simulation.
# Options set the tolerance parameters for the checks.

config = {
    'checks': {
        'structural_checks': [
            'is_single_tree',
            'has_valid_soma',
            'has_soma_points',
            'has_sequential_ids',
            'has_increasing_ids',
            'has_valid_neurites',
            'no_missing_parents',
            'has_all_finite_radius_neurites'
        ],
        'neuron_checks': [
            'has_axon',
            'has_basal_dendrite',
            'has_apical_dendrite',
            'has_no_jumps',
            'has_no_fat_ends',
            'has_nonzero_soma_radius',
            'has_all_nonzero_neurite_radii',
            'has_all_nonzero_section_lengths',
            'has_all_nonzero_segment_lengths',
        ]
    },
    'options': {
        'has_nonzero_soma_radius': 0.0,
        "has_all_nonzero_neurite_radii": 0.007,
        "has_all_nonzero_segment_lengths": 0.01,
        "has_all_nonzero_section_lengths": 0.01,
    }
}

In [ ]:
from neurom.check.runner import CheckRunner

# Create a "CheckRunner" object by providing the configuration dictionary
check_runner = CheckRunner(config)

# Run on a directory or a morphology
# Atention: change path_to_neurons to the path in which you have placed the files
results = check_runner.run(path_to_neurons) 

for morph in results['files']:
    
    print ("Morphology: {} \n".format(morph))
    
    checks_dict = results['files'][morph]
    
    # Empty strings for our results
    passed = ""
    failed = ""
    
    # Iterate over dictionary's items
    for check_name, status in checks_dict.items():
        
        # The string with the check's name and status
        string = "\t{0: <50} : {1}\n".format(check_name, status)
    
        # Separate passed from failed checks
        if status is True:
            
            passed += string # Concatenates to the current passed string
            
        else:
            
            failed += string # Concatenats to the current failed string

    print (passed)
    print (failed)

<font size ="3">As you can see 'bad_morph.asc' has two errors related with segments and sections lengths. Apparently, some of the sections and the segments in this file have a value equal to zero. However, "good_morph.asc" doesn't have any error.